# Develop and evaluate neural network models using Keras for a regression problem

__Author - Sarthak Kaushik__

__Date - October 22, 2021__


In [ ]:
from PIL import Image
import glob
import os
import shutil

import pathlib
import tensorflow as tf

from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D

import warnings
warnings.simplefilter("ignore")

# Image Resizing

In [ ]:
def resize_image(img, path, i):
    # Resize and save image
    image = Image.open(img)
    image = image.resize((150,150))
    image.save(path+'/'+str(i)+'.jpg')
    return
        
def create_directory(path):
    # Create directory for resized images
    if os.path.isdir(path) == True:
        print ("directory exists: " + path)
        print ('deleting directory: ' + path)
        shutil.rmtree(path)
        os.mkdir(path)
        print ("directory created: " + path)
    else :
        os.mkdir(path)
        print ("directory created: " + path)
    return

def resize_images():
    directories = glob.glob('flower_photos/*/')
    path = os.path.join('resized')
    # shutil.rmtree(path)
    create_directory(path)

    for directory in directories:
        flower_type = str.split(directory,'/')[1]    
       # path = os.path.join(path1,'resized',flower_type)
        path2 = os.path.join('resized',flower_type)
        create_directory(path2)
        i = 0
        images= glob.glob(directory+'/*.jpg')
        for image in images:
            resize_image(image, path2 , i)
            i += 1
    shutil.rmtree('flower_photos')
    return

# Split Images Train-Test

In [ ]:
directories = glob.glob('resized/*/')

In [ ]:
data_dir = 'resized'
# data_dir = pathlib.Path(data_dir)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.25,
    subset="training",
    seed=123,
    image_size=(150, 150),
    batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.25,
    subset="validation",
    seed=123,
    image_size=(150, 150),
    batch_size=32)

In [ ]:
class_names = train_ds.class_names
print(class_names)

# Model VGG16 - Top Layer Removed

In [ ]:
image_w, image_h = 150, 150
# VGG16 pre-trained model without fully connected layers and with different input dimensions
modelvgg = tf.keras.applications.VGG16(
    weights = "imagenet", 
    include_top=False, 
    input_shape = (image_w, image_h, 3)
)
modelvgg.summary()

# Functions

## Freezing all layers

In [ ]:
# Freezing all layers we don't want to train
def unfreeze_layers(model, n=0, unfreeze_all = False):
    ''' n : number of layers to freeze'''
    for layer in model.layers:
        layer.trainable = True
    
    # if unfreeze_all == 1 then skip this step
    if unfreeze_all == False:

        # Unfreeze layers as defined by n
        for layer in model.layers[:len(model.layers)-n]:
            layer.trainable = False
    layers = [] 
    for i, layer in enumerate(model.layers): 
            layers.append([layer.name, layer.trainable])
    print (layers)

## Add Fully Connected End Layers

In [ ]:
# Adding custom layers to create a new model 
# one with 256 nodes using ‘relu’ activation and output layer with 5 nodes and ‘softmax’ activation)
# model
def model_custom(model):
    
    model1 = Sequential([
        model,
        Flatten(name='flatten'),
        Dense(256, activation='relu', name='new_fc1', 
              kernel_initializer="HeNormal", 
              kernel_regularizer = regularizers.l2(0.01)),
        Dense(5, activation='softmax', 
              name='new_predictions')
    ])

    print (model1.summary())

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001,)

    model1.compile(
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
        optimizer = optimizer, 
        metrics=["accuracy"])

    return model1

# First Model - No Layers Trainable

In [ ]:
unfreeze_layers(modelvgg, 0)
model1 = model_custom(modelvgg)

In [ ]:
%%time
# Basically tune the weights of the newly added fully connected layers
history = model1.fit(train_ds, validation_data=val_ds, epochs=5)

## Evaluate Model

# Second Model - Last Layer Trainable

## Unfreeze Last Layer

In [ ]:
unfreeze_layers(modelvgg, 4)
model2 = model_custom(modelvgg)

In [ ]:
%%time
# Basically tune the weights of the newly added fully connected layers
history = model2.fit(train_ds, validation_data=val_ds, epochs=5)

## Evaluate Model

# Second Model - All Layers Trainable

## Unfreeze All Layers

In [ ]:
# unfreeze all block
unfreeze_layers(modelvgg, unfreeze_all=True)
model3 = model_custom(modelvgg)

In [ ]:
%%time
# Basically tune the weights of the newly added fully connected layers
history = model3.fit(train_ds, validation_data=val_ds, epochs=5)

## Evaluate Model